In [1]:
import fasttext
import pandas as pd
import numpy as np
import matplotlib as mpl
import nltk,re,pprint
import sys,glob,os
from imp import reload
from nltk.corpus import stopwords

In [2]:
 dataAnalyst = pd.read_csv("../datasets/DataAnalyst.csv", encoding="latin")

In [3]:
len(dataAnalyst["Industry"].unique())

89

In [4]:
dataAnalystSmall = dataAnalyst[['Job Description', 'Company Name', 'Industry']]

In [5]:
dataAnalystSmall

,Job Description,Company Name,Industry
0,Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,Social Assistance
1,Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,Health Care Services & Hospitals
2,Weâre looking for a Senior Data Analyst who ...,Squarespace\n3.4,Internet
3,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,IT Services
4,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,Sports & Recreation
...,...,...,...
2248,Maintains systems to protect data from unautho...,"Avacend, Inc.\n2.5",Staffing & Outsourcing
2249,Position:\nSenior Data Analyst (Corporate Audi...,Arrow Electronics\n2.9,Wholesale
2250,"Title: Technical Business Analyst (SQL, Data a...",Spiceorb,-1
2251,Summary\n\nResponsible for working cross-funct...,Contingent Network Services\n3.1,Enterprise Software & Network Solutions


In [6]:
dataAnalystGrouped = dataAnalystSmall.groupby(['Industry']).count()

In [7]:
pd.set_option('display.max_rows', 500)
print(dataAnalystGrouped.sort_values(by=['Job Description'], ascending=False))

                                           Job Description  Company Name
Industry                                                                
-1                                                     353           352
IT Services                                            325           325
Staffing & Outsourcing                                 323           323
Health Care Services & Hospitals                       151           151
Consulting                                             111           111
Computer Hardware & Software                           111           111
Investment Banking & Asset Management                   78            78
Enterprise Software & Network Solutions                 69            69
Internet                                                65            65
Banks & Credit Unions                                   51            51
Advertising & Marketing                                 51            51
Insurance Carriers                                 

In [8]:
dataAnalystGrouped.shape

(89, 2)

In [9]:
dataAnalystTopCat = dataAnalystSmall[dataAnalystSmall["Industry"].isin(['IT Services', 'Staffing & Outsourcing', 'Health Care Services & Hospitals', 'Consulting',
'Computer Hardware & Software', 'Investment Banking & Asset Management', 'Enterprise Software & Network Solutions', 'Internet',
'Banks & Credit Unions', 'Advertising & Marketing'])]

In [10]:
dataAnalystTopCatGrouped = dataAnalystTopCat.groupby(['Industry'])['Job Description'].apply(lambda x: ','.join(x)).reset_index()

In [11]:
print(dataAnalystTopCatGrouped.shape)
dataAnalystTopCatGrouped.reset_index()

(10, 2)


,index,Industry,Job Description
0,0,Advertising & Marketing,About Known\n\nKnown is a modern marketing com...
1,1,Banks & Credit Unions,"Job Description:\n\nWithin GMC, Data & Portfol..."
2,2,Computer Hardware & Software,"If interested, please reach out to Iram Shah e..."
3,3,Consulting,About Us:\n\nNYSTEC is a non-profit technology...
4,4,Enterprise Software & Network Solutions,"Data Analyst Jersey City, NJ 12+ months contra..."
5,5,Health Care Services & Hospitals,Overview\n\nProvides analytical and technical ...
6,6,IT Services,Requisition NumberRR-0001939\nRemote:Yes\nWe c...
7,7,Internet,Weâre looking for a Senior Data Analyst who ...
8,8,Investment Banking & Asset Management,About Cubist\nCubist Systematic Strategies is ...
9,9,Staffing & Outsourcing,Title: Data Analyst\n\nPosition: 1+ Year Contr...


In [12]:
dataAnalystTopCatGrouped["Job Description"].shape

(10,)

In [25]:
#remove punctuations

class rempunct:
    def __init__(self):
        from nltk.corpus import stopwords
        self.swords = set(stopwords.words('english'))
        print(len(self.swords),"stopwords present!")
    
    def rem_stop_punct(self,originalText, ofilename):
        splittedText = originalText.split()
        lenl = len(splittedText)
        print("Length is: ",lenl, splittedText[:5])
        ofile = open(ofilename,'a')
        
        for r in range(lenl):
            linex = splittedText[r]
            linex2 = "".join(c for c in linex if c not in ('!','.',':',',','?',';','``','&','-','"','(',')','[',']','0','1','2','3','4','5','6','7','8','9'))
            linex3 = linex2.split()
            #prog=(r+1)/len(rawlines)
            for s in range(len(linex3)):
                noword = linex3[s].lower()
                if noword not in self.swords:
                    ofile.write(noword)
                    ofile.write(" ") 

    def drawProgressBar(self,percent, barLen = 50):			#just a progress bar so that you dont lose patience
        sys.stdout.write("\r")
        progress = ""
        for i in range(barLen):
            if i<int(barLen * percent):
                progress += "="
            else:
                progress += " "
        sys.stdout.write("[ %s ] %.2f%%" % (progress, percent * 100))
        sys.stdout.flush()

In [26]:
rp = rempunct()
for index, row in dataAnalystTopCatGrouped.iterrows():
    ofname = '../processFiles/trainCatFiles/cat' + str(index)+ '_' + '-'.join(row['Industry'].split()) + '.txt'    
    print("Processing...",row['Industry'])
    rp.rem_stop_punct(row['Job Description'],ofname)

179 stopwords present!
Processing... Advertising & Marketing
Length is:  21227 ['About', 'Known', 'Known', 'is', 'a']
Processing... Banks & Credit Unions
Length is:  32003 ['Job', 'Description:', 'Within', 'GMC,', 'Data']
Processing... Computer Hardware & Software
Length is:  50038 ['If', 'interested,', 'please', 'reach', 'out']
Processing... Consulting
Length is:  48362 ['About', 'Us:', 'NYSTEC', 'is', 'a']
Processing... Enterprise Software & Network Solutions
Length is:  35454 ['Data', 'Analyst', 'Jersey', 'City,', 'NJ']
Processing... Health Care Services & Hospitals
Length is:  84819 ['Overview', 'Provides', 'analytical', 'and', 'technical']
Processing... IT Services
Length is:  96150 ['Requisition', 'NumberRR-0001939', 'Remote:Yes', 'We', 'collaborate.']
Processing... Internet
Length is:  34604 ['Weâ\x80\x99re', 'looking', 'for', 'a', 'Senior']
Processing... Investment Banking & Asset Management
Length is:  41725 ['About', 'Cubist', 'Cubist', 'Systematic', 'Strategies']
Processing.

In [16]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [62]:
# Perform count
from collections import Counter
import operator
import os
import sys
import string
import argparse
import numpy as np

def getDocumentTF(fname):
    #file1 = open("../processFiles/trainCatFiles/cat2_Computer-Hardware-&-Software.txt","r")
    file1 = open(fname)
    readfile = file1.read()
    wordsInFile = readfile.split()
    counts_all = Counter(wordsInFile)
    words, count_values = zip(*counts_all.items())
    # Sort both lists by frequency in values (Schwartzian transform) - thanks, http://stackoverflow.com/questions/9543211/sorting-a-list-in-python-using-the-result-from-sorting-another-list
    values_sorted, words_sorted = zip(*sorted(zip(count_values, words), key=operator.itemgetter(0), reverse=True))
    # Top N
    top_n = 100
    words_sorted_top = words_sorted[0:top_n]
    values_sorted_top = values_sorted[0:top_n]
    countdf = pd.DataFrame({'count': values_sorted_top, 'word': words_sorted_top})
    return countdf
    
#def processAllDocuments():
owd = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN"
print(owd)
fld = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/"
os.chdir(fld)
arr = []
for file in glob.glob("*.txt"):
    print("found",file)
    doctf = getDocumentTF(file)
    arr.append(doctf)
os.chdir(owd)

/Users/arnabborah/Documents/repositories/textclusteringDBSCAN
found cat5_Health-Care-Services-&-Hospitals.txt
found cat6_IT-Services.txt
found cat0_Advertising-&-Marketing.txt
found cat7_Internet.txt
found cat4_Enterprise-Software-&-Network-Solutions.txt
found cat1_Banks-&-Credit-Unions.txt
found cat3_Consulting.txt
found cat9_Staffing-&-Outsourcing.txt
found cat2_Computer-Hardware-&-Software.txt
found cat8_Investment-Banking-&-Asset-Management.txt


[    count              word
 0    1981              data
 1     704        experience
 2     537            health
 3     397              work
 4     391          business
 5     354          analysis
 6     325        management
 7     324            skills
 8     298           ability
 9     291              care
 10    288           analyst
 11    273       information
 12    271           quality
 13    264              team
 14    264           reports
 15    258         reporting
 16    257          research
 17    256        healthcare
 18    242         knowledge
 19    232         including
 20    225           related
 21    221          required
 22    217             years
 23    214               â¢
 24    209           support
 25    203            degree
 26    203               sql
 27    196          clinical
 28    193             tools
 29    190         preferred
 30    185          position
 31    177      requirements
 32    174         analytics
 33    165    

In [60]:
allDoctfidf

['cat5_Health-Care-Services-&-Hospitals.txt',
 'cat6_IT-Services.txt',
 'cat0_Advertising-&-Marketing.txt',
 'cat7_Internet.txt',
 'cat4_Enterprise-Software-&-Network-Solutions.txt',
 'cat1_Banks-&-Credit-Unions.txt',
 'cat3_Consulting.txt',
 'cat9_Staffing-&-Outsourcing.txt',
 'cat2_Computer-Hardware-&-Software.txt',
 'cat8_Investment-Banking-&-Asset-Management.txt',
 None]

In [61]:
#countdf
#countdf.query('word==data')['count']
#print(type(countdf[countdf['word']=="data"]['count']))

def termfreq(term):
    return countdf[countdf['word']==term]['count']

termfreq("business")

2    405
Name: count, dtype: int64

In [ ]:
#tfidf

class flingTFIDF:
    def __init__(self):
        

    def allfiles(self,foldername):			#returns the name of all files inside the source folder. 		
        owd = os.getcwd()
        fld = foldername + "/"
        os.chdir(fld)					#this is the name of the folder from which the file names are returned.
        arr = []						#empty array, the names of files are appended to this array, and returned.
        for file in glob.glob("*.txt"):
            arr.append(file)
        os.chdir(owd)
        return arr